In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from selenium.webdriver import ActionChains

In [7]:
opts = Options() 
opts.add_argument("user.agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
#para que se abra el navegador en segundo plano
#opts.add_argument('--headless')
opts.add_argument('--no-sandbox')
opts.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), # detecta la version de nuestro navegador
    options= opts
)


In [3]:
"""
    C A R R E F O U R
"""

#Decidimos que productos buscar y cargamos la pagina de busqueda
busqueda = 'chocolate'
#Abrimos la pagina 
driver.get("https://www.carrefour.es/?q="+busqueda.replace(" ","+"))
sleep(2)
#Aceptamos las cookies
driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
sleep(2)
driver.maximize_window()

In [8]:
"""
    C O N S U M 
"""
#Decidimos que productos buscar y cargamos la pagina de busqueda
busqueda = 'chocolate'
#Abrimos la pagina 
driver.get("https://tienda.consum.es/es/s/"+busqueda.replace(" ","%20"))
sleep(2)
#Aceptamos las cookies
driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
sleep(2)
driver.maximize_window()

In [ ]:
"""SCROLLING"""

#Vamos a medir el largo de la página para realizar un scroll hasta el final
last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    #desplaze el documento a la posición vertical del final
    print(last_height)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(3)
    new_height = driver.execute_script('return document.body.scrollHeight')
    #Una vez cargado el final añadimos el nuevo tamaño vertical
    if last_height == new_height:
        break
    else:
        last_height =new_height


3318


In [9]:
"""
    Creamos la clase articulo  e iniciamos la lista Articulos que contendra 
    los objeetos de la clase Articulo como diccionarios
"""
class Articulo:
  def __init__(self, descripcion, precio):
    self.descripcion = descripcion
    self.precio = precio

def obj_dict(obj):
  return obj.__dict__

Articulos = []

In [5]:
"""
    C A R R E F O U R
"""

#instanciamos el objeto beautiful soup para recorrer la pagina
soup = bs(driver.page_source, "lxml")
productos = soup.find('section',{'id':'ebx-grid'}).find_all('article')
for producto in productos:
    try:
        Articulos.append(Articulo(producto.findChild('h1', {'class':'ebx-result-title ebx-result__title'}).text, producto.findChild('strong', {'class':'ebx-result-price__value'}).text))
    except:
        print('valor desconocido')
        #Articulos.append(Articulo(producto.findChild('h1', {'class':'ebx-result-title ebx-result__title'}).text, '--'))

valor desconocido
valor desconocido


In [14]:
"""
    C O N S U M 
"""
#instanciamos el objeto beautiful soup para recorrer la pagina
soup = bs(driver.page_source, "lxml")
productos = soup.find('div',{'class':'ng-star-inserted grid__widget-3'}).find_all('cmp-widget-product')

for producto in productos:
    try:
        precio = producto.findChild('span', {'id':'grid-widget--price'}).text
        print(precio)
    
        Articulos.append(Articulo(producto.findChild('a', {'id':'grid-widget--descr'}).text, precio))
        
    except:
        print('valor desconocido')
        #Articulos.append(Articulo(producto.findChild('a', {'id':'grid-widget--descr'}).text, ""))

1,99 €
3,80 €
1,29 €
1,05 €
1,85 €
2,75 €
1,05 €
2,75 €
4,99 €
3,25 €
3,25 €
3,25 €
3,25 €
2,80 €
4,85 €
3,49 €
2,75 €
3,25 €
2,99 €
2,30 €


In [6]:
"""
    Pasamos los datos extraidos de la web a un fichero
"""

import json

with open('carrefour.txt', 'w+') as fichero:
  fichero.write(json.dumps(Articulos, indent=2, default=obj_dict, ensure_ascii=False))

In [15]:
"""
    Pasamos los datos extraidos de la web a un fichero
"""

import json

with open('consum.txt', 'w+', encoding='utf8') as fichero:
  fichero.write(json.dumps(Articulos, indent=2, default=obj_dict, ensure_ascii=False))

In [18]:
from difflib import SequenceMatcher
import json
import pprint

def from_json(json_object):
    if '__class__' in json_object: 
        if json_object['__ class__'] == 'bytes':
            return bytes(json_object['__value__'])
        return json_object

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

with open('consum.txt', 'r+', encoding='utf8') as fichero:
    listado_consum = json.load(fichero)
    #datos = json.load(fichero, object_hook=from_json)
#print(listado_consum)

with open('carrefour.txt', 'r+') as fichero:
    listado_carrefour = json.load(fichero)
    #datos = json.load(fichero, object_hook=from_json)
#print(listado_carrefour)

#Añadiremos a una lista los resultados
comparador = []
           

def comparacion(lista_consum, lista_carrefour):
    #caso base cunado una de las listas es vacia
    if len(lista_carrefour) == 0:
        for articulo in lista_consum:
            comparador.append({'consum':articulo, 'carrefour':{'-','-'}})
    if len(lista_consum) == 0:
            for articulo in lista_carrefour:
                comparador.append({'consum':{'-':'-'}, 'carrefour':articulo})   

    for i, prod_consum in enumerate(lista_consum):
        for j in range(len(lista_carrefour)):
            similitud = similar(prod_consum['descripcion'], lista_carrefour[j]['descripcion'])
            #Si coinciden dejamos de recorrer el bucle para enlazar el siguiente porducto de consum
            if similitud >= 0.66:
                comparador.append({'consum':prod_consum, 'carrefour':lista_carrefour[j], 'porcentaje': similitud})
                lista_consum.pop(i)
                lista_carrefour.pop(j)
                comparacion(lista_consum, lista_carrefour)
                break
            #si estamos en el ultimo producto y no hemos podido enlazarlo con ninguno
            if j == (len(lista_carrefour)-1):
                comparador.append({'consum':lista_consum[i], 'carrefour':{'-','-'}})
                lista_consum.pop(i)
                comparacion(lista_consum, lista_carrefour)
        # hay dos caminos o se pueden enlazar y entramos ebn otra funcion o
        # el producto consum no se puede enlazar y entramos en otra funcion con ese productos añadido solo
        # entonces despues de haber realizado uno de estos dos caminos no necesitaremos recorrer mas el bucle
        break
comparacion(listado_consum, listado_carrefour)             
#print(json.dumps(comparador))
pprint.pprint(comparador)





[{'carrefour': {'-'},
  'consum': {'descripcion': 'Chocolate Negro 70% sin Azúcar 125 Gr',
             'precio': '1,99\xa0€'}},
 {'carrefour': {'-'},
  'consum': {'descripcion': 'Chocolate Puro 300 Gr', 'precio': '3,80\xa0€'}},
 {'carrefour': {'descripcion': 'Chocolate negro 50% Carrefour 150 g.',
                'precio': '0,78 €'},
  'consum': {'descripcion': 'Chocolate Negro 55% con Almendra 150 Gr',
             'precio': '1,29\xa0€'},
  'porcentaje': 0.6666666666666666},
 {'carrefour': {'-'},
  'consum': {'descripcion': 'Chocolate Negro 55% 150 Gr',
             'precio': '1,05\xa0€'}},
 {'carrefour': {'-'},
  'consum': {'descripcion': 'Chocolate Negro 55% con Avellanas Enteras 200 Gr',
             'precio': '1,85\xa0€'}},
 {'carrefour': {'-'},
  'consum': {'descripcion': 'Chocolate Negro Bio 85% cacao 90 Gr',
             'precio': '2,75\xa0€'}},
 {'carrefour': {'-'},
  'consum': {'descripcion': 'Chocolate Negro 72% Cacao 100 Gr',
             'precio': '1,05\xa0€'}},
 {'carref